In [2]:
%pip install anthropic

  Using cached anthropic-0.42.0-py3-none-any.whl.metadata (23 kB)
Using cached anthropic-0.42.0-py3-none-any.whl (203 kB)
Note: you may need to restart the kernel to use updated packages.


In [497]:
from anthropic import Anthropic
from anthropic import AsyncAnthropic
from dotenv import load_dotenv
import os 
import openai

client = Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))

async_client = AsyncAnthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))

openai_client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def get_completion_openai(messages, temperature=1) -> str: 
    response = openai_client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        max_tokens=4096,
        temperature=temperature, 
    )
    return response.choices[0].message.content

async def get_completion_async(prompt, system_prompt: str = "", tools=[]):
    response = await async_client.messages.create(
        model="claude-3-5-sonnet-20240620",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=4096,
        temperature=1, 
        top_p=0.9,
        system=system_prompt,
        tools=tools,
    )
    return response

def get_completion(messages: list, system_prompt: str = "", tools=[], temperature=1):
    return client.messages.create(
        model="claude-3-5-sonnet-20240620",
        messages=messages,
        max_tokens=8192,
        temperature=temperature, 
        system=system_prompt,
        tools=tools,
    )


In [458]:
import asyncio
import nest_asyncio

nest_asyncio.apply()

story_prompt = """
다음 지침에 따라 이야기를 각색해주세요.
    1. 서술 방식
    - 마치 친구에게 직접 이야기하듯 구어체로 서술
    - 현장감 있는 묘사와 생생한 감정 표현 사용
    - 대화문은 ""로 표시 (예: "안녕하세요")
    - 내적 독백은 ''로 표시 (예: '이게 뭐지?')
    - 상황 설명은 ~했어, ~였어 등 친근한 과거형 사용
    - 첫 문장 작성 규칙: 임팩트 있는 한 줄 요약으로 시작 (e.g "여자친구가 두 명이 됐다", "담임선생님을 꼬셔버렸다.")

    2. 이야기 구조
    - 상황 설정: 언제, 어디서, 누구와 있었는지 구체적으로 시작
    - 사건 전개: 주인공의 내적 감정과 의도를 포함하여 자연스럽고 긴장이 끊이지 않게 리드미컬하게 전개
    - 핵심 장면: 가장 극적인 순간을 자세하게 묘사
    - 결말: 그 순간 이후의 변화나 영향을 포함

    3. 감정선 표현
    - 주인공의 심리 변화를 내적 독백으로 자연스럽게 표현
    - 다른 등장인물들의 반응도 구체적으로 묘사
    - 유머러스한 요소나 반전을 적절히 활용

    4. 현장감 있는 디테일
    - 소리, 동작, 표정 등을 구체적으로 묘사
    - 시간의 흐름을 잘 보여주는 전개
    - 등장인물들의 생생한 반응 포함

    5. 결말 처리
    - 억지로 스토리를 만들어내지 말고 자연스러운 결말을 제시
    - 억지 교훈이나 아이러니 함을 제시하지마

    다음 <example_story_N> 태그들에 있는 이야기를 예시로 참고하면 돼. 
    
    <example_story_1>
      <title> 여친이 복사되는 마법 </title>
      <content> 여자친구가 두 명이 됐다. 와이프가 연애 초반에 자기 언니랑 자취했거든 근데 하루는 “언니가 오빠 궁금하다는데 우리집에 놀러올래?” (대사가 필요한 부분은 “”로 강조) “난 당연히 좋지” 가족분들 중에 처음으로 뵙는거라서 전날에 정장까지 다려놨어. 약속 당일날 과일 사들고 가니까 와이프가 현관문 열어주는데 “오빠 빨리 왔네” 과일 바구니만 넘겨주고 화장실에서 손씻고 나오니까 와이프가 부엌에서 과일을 씻고 있는거야 ‘자기 옷 갈아입었네..?’ (내적 독백은 ‘’ 로 강조) 뒤에서 백허그를 하는 순간 “꺅!!” “왜!!” “뭐야?” 와이프가 내 뒤에서 뛰어오는데 그럼 내가 백허그한 사람은..? 맞아.. 우리 처형이었어. 와이프랑 처형이 일란성 쌍둥인데 헤어스타일도 비슷해서 뒤에서 보면 구분이 안돼 “죄송합니다 00인줄 알았어요” “괜찮아요 저희 부모님도 헷갈려하세요” 처형과의 첫만남은 진짜 흑역사였어. 와이프랑 결혼하고 나서도 처형이랑 같이 술마실때면 제부한테 백허그도 받고 가문의 영광이야” “그만 놀리세요 부끄러워잉” </content< 
    </example_story_2>

    <exmaple_stroy_2> 
      <title> 선생님, 이젠 자기라고 부를게. </title>
      <content> 담임선생님을 꼬셔버렸다. 고3 때 젊은 여자쌤이 담임이 됐는데 내가 남중 남고를 나오기도 했고 그 쌤이 예뻐서 좋다는 티를 엄청 내고다녔어. 오죽하면 다른 쌤들이 "너 00쌤 너무 좋아하는거 아니야?"" 물론 그 선생님은 내가 부담스러우니까 선은 지켜줬으면 좋겠다. 그 말 들은 이후론 나도 반포기 상태로 조용하게 학교다녔지 그렇게 졸업을 하고 바로 군대갔다가 대구에서 취직을 했어. 근데 친구들한테 선생님 소식 들어보니까 대구로 학교를 옮기셨다고 하더라고 선생님에 대한 감정도 사라졌고 그냥 그분이 잘 지내시나 싶어서 톡을 보냈다. “선생님 저 기억하세요?” 근데 어쩌다보니 그 선생님이랑 밥을 먹게 된거야 토요일 점심에 약속장소에서 기다리는데 “00아 잘 지냈어?” 선생님 보는 순간 심장이 쿵 내려앉더라 그날부터 내 짝사랑은 다시 시작됐어 주말마다 밥 먹자고 꼬시고 퇴근할 때 데리러가고 나 진짜 많이 노력했거든 결국엔 선생님이랑 사귀고있다 ㅋㅋ 이젠 선생님 아니고 자기라고 불러 6살 연상이지만 너무 귀엽다 </content> 
    </example_story_2>

    원본 이야기: {story}
"""

async def create_dramatic_narration(story):
    system_prompt = """
    당신은 시나리오를 담당하는 ‘작가/각색가’ 역할로서 사용자(또는 다른 에이전트)로부터 원본 이야기를 입력받아, 숏폼 영상용 시나리오를 최적화하고 완성해야 합니다. 
    
    당신의 목적(Goal)은 다음과 같습니다:
    a) 인터넷 썰을 영상용 시나리오로 재탄생시키는 것이 최종 목적
    b) 핵심 정보 파악 & 요약: 원본 썰의 주제, 갈등, 감정 포인트를 신속히 추출하고, 불필요한 부분은 과감히 정리한다.
    c) 짧고 임팩트 있는 시나리오 완성: 1분 이하의 숏폼 콘텐츠에 맞는 분량으로 압축. 결말(Ending)·반전(Point)·유머 요소를 적절히 배치해 시청자 만족도를 극대화한다.
    d) 장면(Scene)별 구체적 디테일 제시: 각 장면에 필요한 등장인물, 상황, 감정, 대사를 구조적으로 작성해, AI 이미지/영상 생성 및 TTS가 손쉽게 연계될 수 있도록 한다.
    e) 톤 & 스타일 관리: 스토리를 참고해 시나리오 전반의 분위기와 문체를 유연하게 조절한다. 저작권, 개인정보 등 민감한 요소는 배제하거나 각색해, 문제 없는 콘텐츠를 만든다.
  
    당신의 backstory는 다음과 같습니다:
    a) 이 에이전트는 수많은 소설, 시나리오, 스토리텔링 자료를 학습한 대형 언어 모델에 기반하여, 서사 구조와 스토리텔링 기법에 능통하다.
    b) 다양한 장르(코믹, 공포, 드라마, 로맨스)와 매체(웹툰, 영화, 숏폼 영상 등)에 대한 광범위한 학습 데이터를 보유하고 있다.
    c) 틱톡, 유튜브 쇼츠 등 짧은 영상 플랫폼에서 통하는 전개 속도, 대사 호흡, 결말 임팩트의 중요성을 잘 이해하고 있다.
    d) 시청자의 주의 집중 시간을 최대한 짧게 가정하고, 순간적으로 흥미를 끌어야 한다는 점을 알기에, 스토리를 빠르고 명확하게 정리하는 방식을 선호한다.
    e) 원본 썰에서 발생할 수 있는 부족한 서사, 논리적 비약, 밋밋한 결말 등을 창의적 아이디어로 보완해, 완성도 있는 시나리오를 만드는 데 특화되어 있다.
    f) 반전 요소, 개그 포인트 등 감칠맛을 더해, 짧은 영상에서도 시청자가 “오!” 하고 놀라거나 웃음을 터뜨릴 수 있도록 돕는다.
    """    
    
    prompt = story_prompt.format(story=story)
    
    async def generate_stories():
        tasks = [
            get_completion_async(prompt, system_prompt)
            for _ in range(5)  # 5개의 스토리 생성 (1~5)
        ]
        responses = await asyncio.gather(*tasks)
        return [response.content[0].text for response in responses]
    
    dramatic_narrations = await generate_stories()  # 최적의 스토리 선택을 위한 평가 프롬프트
    
    evaluation_prompt = f"""
    다음은 같은 원본 이야기를 바탕으로 각색된 여러 버전의 스토리입니다. 
    각 스토리를 다음 기준으로 1-10점으로 평가하고, 가장 높은 점수의 스토리를 선택해주세요:

    평가 기준:
    1. 숏폼 적합성 (1분 이내 영상화 가능성)
    2. 스토리 구조의 완성도
    3. 감정선과 몰입도
    4. 현장감과 디테일
    5. 결말의 임팩트

    원본 스토리: {story}

    스토리 버전들:
    {[f"=== 버전 {i+1} ===\n{narr}\n" for i, narr in enumerate(dramatic_narrations)]}

    다음 XML 형식으로 응답해주세요:
    <result>
        <version_1>
            <scores>
                <shortform>점수</shortform>
                <structure>점수</structure>
                <emotion>점수</emotion>
                <detail>점수</detail>
                <ending>점수</ending>
                <total>총점</total>
            </scores>
            <comment>간단한 평가</comment>
        </version_1>
        <version_2>
            <scores>
                <shortform>점수</shortform>
                <structure>점수</structure>
                <emotion>점수</emotion>
                <detail>점수</detail>
                <ending>점수</ending>
                <total>총점</total>
            </scores>
            <comment>간단한 평가</comment>
        </version_2>
        ...
        <selected_version>선택된 버전 번호</selected_version>
        <reason>선택 이유</reason>
    </result>
    """
    
    # 평가 결과 얻기
    response = await get_completion_async(evaluation_prompt)  # ✅ 권장
    evaluation = response.content[0].text
    try:
        # XML 문자열에서 selected_version 태그 사이의 값을 추출
        selected_version = int(
            evaluation.split('<selected_version>')[1].split('</selected_version>')[0]
        ) - 1
    except:
        # 파싱 실패시 첫 번째 버전 선택
        print("파싱 실패")
        selected_version = 0
        
    return dramatic_narrations[selected_version], evaluation
    

In [484]:
from textwrap import dedent
story = dedent("""
직접 경험한 여초팬 개 진상 썰

작년 9월인가 10월인가 대구에서 한 페발이었는데 이게 규모가 엄청 컸음

라인업이 페발 중 1티어 급이었는데 그 중에 T** ***s 라는 보이그룹이 있었음

어느정도의 인기인지는 모르겠는데 꽤나 굉장하더라고

내가 찍덕이라 좋은자리 먹으려고 공연 전날 다님

이 때도 공연 전날 이른 오후에 갔음

이미 이 팬덤이 꽤나 많이 와있더라고 

저녁 시간 되고 사람들 슬슬 오기 시작하니까 그 팬덤중 한명이 총대메고 줄세우기 시작함

(총대가 뭐냐면 쉽게 말해 임시 반장같은거임. 이게 아주 피곤하고 힘든 일이라 총대 멘다고 표현함)

여튼 그렇게 줄을 서고 있는데 문제가 여기서 발생함

밤 10시쯤이었나 

이미 줄이 꽤 길어져서 내 뒤로 쉽게 세기 힘들정도가 됐는데 갑자기 누군가 줄을 하나 더 만듦ㅋㅋ

뒤에 서기 싫으니까 그냥 아예 지들이 줄을 하나 더 만들어버림

이때부터 전쟁시작 

줄 인원으로만 따지면 거의 200:30? 수준이었는데 이게 사람마음이 ㅈㄴ 간사한게 늦게 온 애들이 줄 길어진거 보고 짧은쪽으로 ㅈㄴ 감

비도오고 전날부터 와서 체력적으로도 ㅈㄴ 힘든데 이제 멘탈까지 박살나기 시작함

양 그룹의 총대끼리 ㅈㄴ 싸우고 지랄이남

그렇게 아침이 됐는데 

줄이 거의 6.5 : 3.5 ?수준까지 따라잡힘

사실상 둘다 길어서 비율이 무의미한 느낌이 들정도까지 됐음

근데 진짜 무서운게 정치랑 선동을 하더라고

줄 두개 중 하나가 메인이 되어야 하는 상황

나중에 줄 만든 또라이년이 일부러 다같이 환호성 지름 (본인들 줄이 메인줄 된 척해서 사람들 끌어모으려고 한마디로 블러핑)

비오고 길 개미끄러운데 갑자기 그 환호성듣고 사람들 존나 뛰쳐나감 넘어지고 다치고 난리남

트위터랑 페북같은데다가 지금 상황 어떤식인지 실시간으로 정치하고 ㅋㅋ (아이돌판은 트위터가 대세)

결국 줄 두개 다 인정 받고 번갈아가면서 들어가기로 합의함 

결국 걔네들 입장에선 절반의 성공한셈이지

근데 ㅈㄴ 어이가없는게 둘이 같은 팬덤임

한마디로 내전한거

진짜 겪은 썰 다 풀려면 하루죙일도 모자름

그리고 그 썰들의 주인공은 100% 전부 여자팬임           
""") 


dramatic_narration, evaluation = await create_dramatic_narration(story)  # ✅
print(dramatic_narration)

"팬덤의 전쟁, 줄 세우기로 시작된 아이돌 콘서트장의 아수라장"

작년 가을, 대구에서 열린 대형 페스티벌에 갔어. T** ***s라는 인기 보이그룹이 출연한다길래 찍덕인 나는 좋은 자리를 잡으려고 공연 전날부터 현장에 도착했지.

오후부터 이미 팬들이 모여들기 시작했는데, 저녁 무렵엔 한 팬이 총대를 메고 줄 세우기를 시작했어. '드디어 질서가 잡히는구나' 싶었지.

그런데 밤 10시쯤, 예상치 못한 일이 벌어졌어. 

"야, 저기 봐. 누가 줄을 하나 더 만들고 있어!"

뒤를 돌아보니 정말 누군가가 새로운 줄을 만들고 있더라고. 늦게 와서 긴 줄 뒤에 서기 싫었나 봐. 

'이게 뭐야, 말도 안 돼!' 

순간 현장이 아수라장으로 변했어. 원래 줄에 있던 200명과 새 줄의 30명 사이에 전쟁이 시작된 거지.

"우리가 먼저 왔잖아!", "그래도 우리도 기다렸다고!"

밤새 비가 내리는 가운데 양측 총대들이 격렬하게 싸우는 모습을 보며 '아, 이게 바로 팬덤의 민낯이구나'라는 생각이 들더라.

아침이 되자 상황은 더 복잡해졌어. 새 줄이 점점 길어져서 6.5:3.5 정도로 따라잡혔거든. 

그때, 한 무리가 갑자기 환호성을 지르기 시작했어.

"우리 줄이 메인이래! 빨리 이쪽으로 와!"

'뭐야, 저게 진짜야?' 

순식간에 사람들이 우르르 몰려들었고, 미끄러운 길에서 넘어지고 밀치는 아비규환이 벌어졌어.

트위터에선 실시간으로 상황 중계가 올라오고, 누가 먼저 왔는지, 어느 줄이 정당한지 논쟁이 불붙었지.

결국 두 줄 다 인정받고 번갈아 입장하기로 했는데, 가장 어이없었던 건...

"야, 우리 같은 팬덤 아니야?"

그래, 모두가 같은 팀을 응원하는 팬들이었던 거야. 내전이었다고 할까?

그날 밤 집에 돌아와 침대에 누워 생각했어. 
'오늘 본 건 단순한 줄 싸움이 아니었어. 이건 인간의 본성과 집단심리를 적나라하게 보여준 거야.'

그리고 한 가지 더, 모든 주인공이 여자 팬이었다는 거. 아이돌 팬덤의 세계는 참 무서운 곳이야.


In [485]:
from textwrap import dedent

def create_title(dramatic_narration):
    prompt = dedent(f"""
    당신은 시나리오를 바탕으로 사람들의 어그로를 끌고 조회수를 올릴 수 있는 제목을 생성해주는 역할을 해야합니다. 
    
    10개 정도의 제목을 생성해주세요. 
    
    시나리오는 다음과 같습니다: 
    {dramatic_narration}
    """)
    
    messages = [{"role": "user", "content": prompt}]
    response = get_completion(messages).content[0].text
    return response

titles = create_title(dramatic_narration)
print(titles)

1. "아이돌 콘서트장 대란! 팬들의 '줄 세우기 전쟁' 현장 포착"

2. "충격! 아이돌 팬덤 내전, 줄 세우기로 벌어진 아수라장"

3. "팬심의 배신? 콘서트 줄 세우기에서 벌어진 충격적 사건"

4. "온라인 폭발! 아이돌 콘서트 줄 세우기 논란의 진실"

5. "여성 팬덤의 민낯? 콘서트장 줄 세우기 대란의 전말"

6. "눈물의 밤샘 대기! 아이돌 콘서트 입장 줄 대란 실화?"

7. "팬덤 전쟁의 서막, 콘서트 줄 세우기에서 벌어진 충격적 사건"

8. "아이돌 사랑? 아니, 전쟁! 콘서트장 줄 세우기 대란 현장 고발"

9. "팬심 vs 이기심? 아이돌 콘서트 줄 세우기 논란의 이면"

10. "충격 폭로! 아이돌 콘서트장에서 벌어진 '줄 세우기 전쟁'의 실체"


In [486]:
title = "여성 팬덤의 민낯? 직접 경험한 여초팬 개 진상 썰"

In [487]:
import json 
import hashlib
import os
from typing import Dict, Set, List

class DuplicateChecker: 
    def __init__(self):
        self.hash_cache: Dict[str, Set[str]] = {
            "story_generation": set(),
            "adapt_story": set(),
        }
        self._loading_existing_hashes()
        
    def _loading_existing_hashes(self):
        if os.path.exists("fine_tune/data/story_generation.jsonl"):
            with open("fine_tune/data/story_generation.jsonl", "r") as file:
                for line in file:
                    data = json.loads(line)
                    content = data["messages"][1]["content"]
                    self.add_hash("story_generation", content)
        
        if os.path.exists("fine_tune/data/adapt_story.jsonl"):
            with open("fine_tune/data/adapt_story.jsonl", "r") as file:
                for line in file:
                    data = json.loads(line)
                    content = data["messages"][1]["content"]
                    self.add_hash("adapt_story", content)

    def _get_hash(self, value: str) -> str:
        return hashlib.md5(value.encode('utf-8')).hexdigest()

    def add_hash(self, key: str, value: str):
        self.hash_cache[key].add(self._get_hash(value))

    def check_duplicate(self, key: str, value: List[Dict]) -> bool:
        content_hash = self._get_hash(value[1]["content"])
        return content_hash in self.hash_cache[key]


def oneline(text):
    # 1. 먼저 라인 정리
    lines = text.splitlines()
    cleaned_lines = [line.strip() for line in lines]
    text = "\n".join(cleaned_lines)
    return text

def create_story_data(dramatic_narration):
    # 스토리 생성 데이터 생성 
    prompt = dedent(f"""
    당신은 LLM 이 더 나은 스토리 생성을 하기 위해 파인튜닝(Fine-tuning) 데이터를 생성하는 역할을 합니다. 
    
    다음 스토리에서 한 문장의 주요 포인트 부분을 뽑아내서 바탕으로 이런 스토리를 생성할 수 있도록 프롬프트 지시문을 생성해주세요. 
    
    주의사항: 
    - 생성되는 스토리는 유튜브 쇼츠에 사용될 것이므로 사람들의 관심을 끌 수 있어야 합니다. 이를 중심으로 스토리에서 주요 포인트를 뽑아내보세요.  
    - 주요 포인트로 너무 자세하게 뽑아내지 마세요. 그거는 스토리와 큰 차이가 없습니다. 정말 핵심을 담은 한 문장으로 뽑아내세요.
    
    먼저 스토리 입니다: 
    {dramatic_narration} 
    
    이제 프롬프트 지시문 입니다. 바로 프롬프트를 설계해보세요: 
    """)
    
    messages = [{"role": "user", "content": prompt}]
    response = get_completion_openai(messages, temperature=0)
    return response
    

def create_fine_tune_data(story, dramatic_narration):
    checker = DuplicateChecker()
    
    story_generation_prompt = create_story_data(dramatic_narration)
    
    story_generation_message = [
        {"role": "user", "content": oneline(story_generation_prompt)}, 
        {"role": "assistant", "content": oneline(dramatic_narration)}
    ]        
    if not checker.check_duplicate("story_generation", story_generation_message):
        with open('fine_tune/data/story_generation.jsonl', 'a') as f:
            f.write(json.dumps({"messages": story_generation_message}, ensure_ascii=False) + "\n")
            
    adapt_story_message = [
        {"role": "user", "content": oneline(story_prompt.format(story=story))},
        {"role": "assistant", "content": oneline(dramatic_narration)}
    ]
    if not checker.check_duplicate("adapt_story", adapt_story_message):
        with open('fine_tune/data/adapt_story.jsonl', 'a') as f:
            f.write(json.dumps({"messages": adapt_story_message}, ensure_ascii=False) + "\n")
        
create_fine_tune_data(story, dramatic_narration)

In [498]:
def create_scene(dramatic_narration):
    system_prompt = """
    당신은 시나리오를 바탕으로 장면(Scene)들을 상세히 묘사해주는 역할을 해야합니다. 
    
    이 장면(Scene)은 유튜브 쇼츠 영상에 사용될 것이니 영상 편집에 적합한 형식으로 작성해주세요. 
    
    그리고 이 장면(Scene)을 바탕으로 이후에 이미지 프롬프트를 생성할 것이니 상세하고 구체적으로 묘사해주세요.
    
    Scene을 작성할 때, 아래의 항목들을 반드시 포함해주세요:

    1) [Scene 번호]
    - “Scene 1”로 표시.

    2) [시간과 배경]
    - 이 장면이 언제 (아침, 저녁, 특정 연도/계절 등) 벌어지는지
    - 전반적인 배경(도시, 시골, 학교, 회사 등)

    3) [구체적인 장소 묘사]
    - 이 장면이 이루어지는 공간(예: 교실, 사무실, 거리)의 구체적 디테일
    - 주변에 보이는 사물, 환경적 특징, 소리, 조명 등

    4) [장소의 분위기와 감정선]
    - 이 공간이 주는 전반적인 느낌(따뜻한, 긴장된, 생동감 있는 등)
    - 캐릭터들이 느끼는 감정 혹은 현장 분위기

    5) [등장인물]
    - 각 인물의 이름(또는 호칭), 성격, 감정 상태
    - 외형 묘사(의상, 헤어스타일, 표정 등)
    - 이 장면에서 인물들이 수행하는 주요 행동

    6) [이 장면에서 벌어지는 사건과 강조할 주요 포인트]
    - 스토리적으로 중요한 사건(갈등, 해프닝, 대화 등)
    - 시청자가 주목해야 할 포인트(웃음 요소, 반전 가능성, 긴장 요소 등)

    7) [카메라 시점]
    - 이 장면을 영상으로 찍는다면, 어떤 시점에서 촬영하는지(예: 1인칭, 3인칭, 줌인/줌아웃, 패닝 등)
    - 장면이 시작될 때와 끝날 때 카메라가 어떻게 움직이는지
    
    8) [내레이션]
    - 이 Scene 에서 포함될 내레이션들
    """
    
    tools = [
        {
            "name": "create_scene", 
            "description": "장면을 상세히 묘사해야 합니다.",
            "input_schema": {
                "type": "object", 
                "properties": {
                    "scenes": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "scene_number": {"type": "string"},
                                "time_and_background": {"type": "string"},
                                "specific_location_description": {"type": "string"},
                                "location_atmosphere_and_emotion": {"type": "string"},
                                "characters": {"type": "string"},
                                "event_and_key_points": {"type": "string"},
                                "camera_angle": {"type": "string"},
                                "narrations": {"type": "string"},
                            }
                        }
                    }
                },
                "required": ["scenes"]
            }
        }
    ]
    
    prompt = f"""
    다음 시나리오를 바탕으로 유튜브 쇼츠에 사용될 수 있도록 연속적인 여러개의 장면(Scene)들로 상세히 묘사해주세요.
    
    주의사항: 
    - 시나리오에 있던 모든 스토리 요소를 누락하지 말고 Scene 으로 만들어주세요. (스토리에 있는 내용이 모두 Scene 들의 narrations 에 포함되어야 합니다.)
    
    이제 시나리오 입니다: 
    {dramatic_narration}
    """
    
    # if len(dramatic_narration) > 1000: 
        
    
    messages = [{"role": "user", "content": prompt}]
    
    response = get_completion(messages, system_prompt, tools)
    
    scenes = None
    for content in response.content: 
        if content.type == "tool_use" and content.name == "create_scene":
            scenes = content.input["scenes"]
            break
            
    return scenes

In [499]:
def print_scene(scene):
    print("Scene number: ", scene["scene_number"])
    print("Time and background: ", scene["time_and_background"])
    print("Specific location description: ", scene["specific_location_description"])
    print("Location atmosphere and emotion: ", scene["location_atmosphere_and_emotion"])
    print("Characters: ", scene["characters"])
    print("Event and key points: ", scene["event_and_key_points"])
    print("Camera angle: ", scene["camera_angle"])
    print("Narrations: ", scene["narrations"])
    print("-"*100)

scenes = create_scene(dramatic_narration)

for scene in scenes:
    print_scene(scene)


Scene number:  Scene 1
Time and background:  작년 가을, 대구에서 열리는 대형 페스티벌 전날 오후
Specific location description:  페스티벌 현장 입구. 넓은 광장과 그 주변으로 펼쳐진 도시 풍경. 멀리 무대 설치 작업이 진행 중인 모습이 보인다. 입구 주변으로 팬들이 모여들기 시작하고 있다.
Location atmosphere and emotion:  기대감과 설렘이 가득한 분위기. 팬들의 흥분된 목소리와 웃음소리가 들린다.
Characters:  주인공: 20대 초반 여성, 카메라를 든 찍덕(팬사진사). T-shirts와 청바지를 입고 있으며, 긴 머리를 단정하게 묶었다. 표정은 기대에 차 있다.
다른 팬들: 다양한 연령대의 여성들, 대부분 캐주얼한 옷차림
Event and key points:  주인공이 페스티벌 현장에 도착하여 주변을 둘러보며 상황을 파악한다. 팬들이 하나둘 모여들기 시작하는 모습을 관찰한다.
Camera angle:  3인칭 시점. 먼저 페스티벌 현장 전체를 보여주는 와이드 샷으로 시작해 서서히 주인공에게 줌인한다.
Narrations:  작년 가을, 대구에서 열린 대형 페스티벌에 갔어. T** ***s라는 인기 보이그룹이 출연한다길래 찍덕인 나는 좋은 자리를 잡으려고 공연 전날부터 현장에 도착했지.
----------------------------------------------------------------------------------------------------
Scene number:  Scene 2
Time and background:  같은 날 저녁, 해가 지기 시작하는 시간
Specific location description:  페스티벌 입구 앞. 팬들이 줄을 서기 시작한 모습. 주변에 가로등이 하나둘 켜지기 시작한다.
Location atmosphere and emotion:  기대감과 약간의 긴장감이 섞인 분위기. 팬들 사이에서 소곤거리는 소리와 웃

In [500]:
def create_clips(scenes): 
    system_prompt = """
    당신은 장면(Scene)들을 바탕으로 클립(Clip)들을 상세히 묘사해주는 역할을 해야합니다. 
    
    이 클립(Clip)은 유튜브 쇼츠 영상에 사용될 것이니 영상 편집에 적합한 형식으로 작성해주세요. 
    
    그리고 이 클립(Clip)을 바탕으로 이후에 이미지 프롬프트를 생성할 것이니 상세하고 구체적으로 묘사해주세요.
    """
    
    tools = [ 
        {
            "name": "create_clip",
            "description": "클립을 상세히 묘사해야 합니다.",
            "input_schema": {
                "type": "object",
                "properties": {
                    "clips": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "scene_number": {"type": "string", "description": "장면 번호"},
                                "clip_number": {"type": "string", "description": "클립 번호"},
                                "time_and_background": {"type": "string", "description": "클립이 언제 벌어지는지"},
                                "specific_location_description": {"type": "string", "description": "클립이 이루어지는 공간의 구체적 디테일"},
                                "characters": {"type": "string", "description": "클립에 등장하는 등장인물들"},
                                "detailed_character_description": {"type": "string", "description": "등장인물들의 상세한 묘사 (e.g 의상, 표정, 동작, 감정 등)"},
                                "main_action": {"type": "string", "description": "캐릭터의 주요 행동"},
                                "camera_angle": {"type": "string", "description": "카메라 시점"},
                                "location_atmosphere_and_emotion": {"type": "string", "description": "장소의 분위기와 감정선"},
                                "event_and_key_points": {"type": "string", "description": "사건의 주요 포인트"},
                                "narrations": {"type": "string", "description": "클립에 포함될 내레이션들"},
                            },
                        },
                        "required": [
                            "scene_number",
                            "clip_number",
                            "time_and_background",
                            "specific_location_description",
                            "characters",
                            "detailed_character_description",
                            "main_action",
                            "camera_angle",
                            "location_atmosphere_and_emotion",
                            "event_and_key_points",
                            "narrations"
                        ]
                    },
                },
                "required": ["clips"],
            },
        }
    ]
    
    clips = []
    for scene in scenes:
        prompt = f"""
        다음 Story 를 가지고 장면(Scene)들을 바탕으로 유튜브 쇼츠에 사용될 수 있도록 연속적인 여러개의 클립(Clip)들로 상세히 묘사해주세요.
        
        클립(Clip)은 Scene 에 있는 최소 하나의 내레이션을 포함해야 하며, 최대 3개의 문장의 내레이션까지 포함해야합니다. (마침표를 기준으로 판단하면 문장의 개수를 판단할 수 있습니다.)
        
        만들어지는 클립(Clip)은 절대 Scene 의 범위를 벗어나면 안됩니다. 그리고 Scene 에서 누락된 Clip 도 있으면 안됩니다.
        
        Sceene의 범위는 narrations 과 Event and key points 를 보면 알 수 있습니다. 
        
        먼저 스토리 입니다 Story: 
        <story> 
        {dramatic_narration}
        </story>
        
        이제 장면(Scene)입니다: 
        <scene>
        {scene}
        </scene>
        """
        
        messages = [{"role": "user", "content": prompt}]
        response = get_completion(messages, system_prompt, tools, temperature=0)
        
        for content in response.content:
            if content.type == "tool_use" and content.name == "create_clip":
                clips.append(content.input["clips"])
                break   
    
    return clips

In [501]:
def print_clip(clip):
    print("Scene number: ", clip["scene_number"])
    print("Clip number: ", clip["clip_number"])
    print("Time and background: ", clip["time_and_background"])
    print("Specific location description: ", clip["specific_location_description"])
    print("Characters: ", clip["characters"])
    print("Detailed character description: ", clip["detailed_character_description"])
    print("Main action: ", clip["main_action"])
    print("Camera angle: ", clip["camera_angle"])
    print("Location atmosphere and emotion: ", clip["location_atmosphere_and_emotion"])
    print("Event and key points: ", clip["event_and_key_points"])
    print("Narrations: ", clip["narrations"])
    print("-"*100)

clips = create_clips(scenes)

for i, clip_list in enumerate(clips):
    print("Scene:")
    print_scene(scenes[i])
    for clip in clip_list:
        print_clip(clip)

Scene:
Scene number:  Scene 1
Time and background:  작년 가을, 대구에서 열리는 대형 페스티벌 전날 오후
Specific location description:  페스티벌 현장 입구. 넓은 광장과 그 주변으로 펼쳐진 도시 풍경. 멀리 무대 설치 작업이 진행 중인 모습이 보인다. 입구 주변으로 팬들이 모여들기 시작하고 있다.
Location atmosphere and emotion:  기대감과 설렘이 가득한 분위기. 팬들의 흥분된 목소리와 웃음소리가 들린다.
Characters:  주인공: 20대 초반 여성, 카메라를 든 찍덕(팬사진사). T-shirts와 청바지를 입고 있으며, 긴 머리를 단정하게 묶었다. 표정은 기대에 차 있다.
다른 팬들: 다양한 연령대의 여성들, 대부분 캐주얼한 옷차림
Event and key points:  주인공이 페스티벌 현장에 도착하여 주변을 둘러보며 상황을 파악한다. 팬들이 하나둘 모여들기 시작하는 모습을 관찰한다.
Camera angle:  3인칭 시점. 먼저 페스티벌 현장 전체를 보여주는 와이드 샷으로 시작해 서서히 주인공에게 줌인한다.
Narrations:  작년 가을, 대구에서 열린 대형 페스티벌에 갔어. T** ***s라는 인기 보이그룹이 출연한다길래 찍덕인 나는 좋은 자리를 잡으려고 공연 전날부터 현장에 도착했지.
----------------------------------------------------------------------------------------------------
Scene number:  Scene 1
Clip number:  Clip 1
Time and background:  작년 가을, 대구에서 열리는 대형 페스티벌 전날 오후
Specific location description:  페스티벌 현장 입구. 넓은 광장과 그 주변으로 펼쳐진 도시 풍경. 멀리 무대 설치 작업이 진행 중인 모습이 보인다.
Characters:  주인공: 20대 초반 여성

In [512]:
import re
from textwrap import dedent
import json

def flatten_array(arr):
    """중첩 배열을 단일 배열로 평탄화하는 함수"""
    result = []
    for item in arr:
        if isinstance(item, list):
            result.extend(flatten_array(item))  # 재귀적으로 중첩 배열 처리
        else:
            result.append(item)
    return result

def extract_image_prompt(text):
    pattern = r'<image_prompt>(.*?)</image_prompt>'
    matches = re.findall(pattern, text, re.DOTALL)  # re.DOTALL flag allows matching across multiple lines
    return matches[0].strip() if matches else None

def create_image_prompt(clips, scenes):
    messages = []
    image_prompts = []
    
    try: 
        with open("fine_tune/data/image_generation_prompt.jsonl", "a", encoding="utf-8") as f:
            clip_index = 0
            for scene in scenes:
                scene_number = int(scene["scene_number"].split("Scene ")[1].strip()) - 1 
                for clip in clips[scene_number]:
                    prompt = dedent(f"""
                    다음 클립(Clip)을 바탕으로 이미지 프롬프트를 생성해 주세요. 
                    
                    이미지 프롬프트 생성을 위한 가이드라인은 다음과 같습니다:

                    1. 입력 분석
                    - Scene과 Clip을 분석하여 이미지의 상황을 파악하세요. clip 에 있는 event_and_key_points와 narrations 참고하면 도움이 됩니다. 
                    - 분석 결과는 <analyze> 태그 안에 포함시켜 주세요. 

                    2. 이미지 프롬프트 구성 요소:
                    A. 이미지 프롬프트 구조화: 
                        - [이미지 스타일]
                        - [캐릭터 설명] (특징, 외모, 의상)
                        - [행동/표정 묘사]
                        - [구도 설명] 
                        - [배경 설명]
                        - [Negative Prompt] (이미지에서 하면 안되는 사항을 명시. 예를 들면 캐릭터는 카메라를 보면 안된다 등)
                    
                    A. 이미지 스타일: 
                        - 이미지의 미적 방향성을 결정하는 스타일은 Japanese anime-style 입니다. 
                    
                    B. 캐릭터 설계 (최대 2명)
                        - 캐릭터 수 선언 (예: "There are 2 characters")
                        - 각 캐릭터별 상세 정보:
                            * 식별 가능한 이름 정의 (여러 캐릭터를 입력하면 섞이므로, First Character [The Nervous Student], Second Character [The Bully] 이런식으로 설계하면 됨)
                            * 나이/성별
                            * 표정 (강조해서 묘사)
                            * 의상 (색상 명시)
                            * 헤어 스타일
                            * 자세/몸짓 (구체적 묘사)
                            * 키
                            * 행동
                        - 캐릭터 별 명확히 구별이 되도록 특성을 강조하세요.
                        
                    C. 캐릭터 행동 설계
                        - clip 에 있는 event_and_key_points, main_action, detailed_character_description 참고하면 어떤 행동을 해야하는지 이해하는데 도움이 됩니다. 
                        - 행동은 1차원적으로 설명을 해야합니다. 그러니까 추상적인 표현 대신 구체적인 수치와 방향을 사용해야 합니다. (예: 위협을 가하는 모습으로 그려줘 대신에 오른쪽 팔을 80도로 올리고 왼쪽 팔로 A 캐릭터의 어깨를 잡고 있다 이런 식으로 묘사해야 합니다.)
                        - 전체적 자세 (body posture) → 구체적 행동(action) → 표정(expression) 순서로 묘사해야 합니다. 
                        - 행동을 받는 대상을 명확히 지정하세요. 
                        - 캐릭터 간 상호작용 강조해주세요.

                    D. 구도 설계: 
                        - 카메라의 각도와 카메라에 캐릭터들이 어떻게 배치되는지 명확히 설계해주세요. 
                        - 캐릭터들간의 좌/우 위치 관계
                        - 캐릭터들간의 거리 
                        - 시점과 각도 

                    D. 배경 설계
                        - Scene의 time_and_background 정보를 활용하면 배경을 명확하게 설계하는데 도움이 될겁니다. 
                        - 간략하게 한 문장으로 통합해주세요.

                    3. 주의사항
                    - 최대 600자 제한
                    - 시간적 순서에 맞게 이전 이미지와의 일관성 유지 (의상, 배경 등)
                    - 행동으로 인한 변화보다 현재 상태 묘사
                    
                    4. 출력 형식
                    - 문장은 마침표(.)로 구분
                    - 같은 범주 내 설명은 마침표 없이 연결
                    - 이미지 프롬프트는 영어로 작성하여 <image_prompt> 태그 안에 포함

                    입력:
                    
                    먼저 장면(Scene) 입니다: 
                    <scene>
                    {scene}
                    </scene>

                    이제 클립(Clip) 입니다: 
                    <clip>
                    {clip}
                    </clip>
                    """) 
                                    
                    prompt = oneline(prompt)
                    
                    messages.append({"role": "user", "content": prompt})
                    response = get_completion_openai(messages, temperature=0)
                    
                    image_prompt = extract_image_prompt(response)
                    image_prompts.append(image_prompt)
                    
                    print(f"Clip number: {clip_index}, clip: {clip}")
                    print(image_prompt)
                    print("-"*100)
                    
                    messages.append({"role": "assistant", "content": response})
                    
                    fine_tune_data = {
                            "messages": [
                                {"role": "user", "content": prompt},
                                {"role": "assistant", "content": response}
                            ]
                    }
                    
                    clip_index += 1
                    f.write(json.dumps(fine_tune_data, ensure_ascii=False) + "\n")
    except PermissionError:
        print("파일 쓰기 권한이 없습니다.")
    
    except Exception as e:
        print(f"파일 쓰기 중 오류 발생: {e}")
    
    return image_prompts, messages

In [514]:
image_prompts, messages = create_image_prompt(clips, scenes)

for i, image_prompt in enumerate(image_prompts):
    print(image_prompt.strip().replace("\n", " "))
    for message in messages[i:2*(i+1)]:
        print(message)
    print("---------------------------------")

Clip number: 0, clip: {'scene_number': 'Scene 1', 'clip_number': 'Clip 1', 'time_and_background': '작년 가을, 대구에서 열리는 대형 페스티벌 전날 오후', 'specific_location_description': '페스티벌 현장 입구. 넓은 광장과 그 주변으로 펼쳐진 도시 풍경. 멀리 무대 설치 작업이 진행 중인 모습이 보인다.', 'characters': '주인공: 20대 초반 여성, 카메라를 든 찍덕(팬사진사)', 'detailed_character_description': '주인공은 T-shirts와 청바지를 입고 있으며, 긴 머리를 단정하게 묶었다. 표정은 기대에 차 있고, 손에는 고급 DSLR 카메라를 들고 있다. 어깨에는 카메라 가방을 메고 있다.', 'main_action': '주인공이 페스티벌 현장에 도착하여 주변을 둘러보며 상황을 파악한다.', 'camera_angle': '와이드 샷. 페스티벌 현장 전체를 보여주는 넓은 앵글', 'location_atmosphere_and_emotion': '기대감과 설렘이 가득한 분위기. 멀리서 들리는 공사 소리와 함께 팬들의 흥분된 목소리가 들린다.', 'event_and_key_points': '주인공이 페스티벌 현장에 도착하여 주변을 둘러보며 상황을 파악한다.', 'narrations': '작년 가을, 대구에서 열린 대형 페스티벌에 갔어.'}
[Japanese anime-style] There is 1 character. First Character [The Enthusiastic Photographer]: Female, early 20s. Expression: full of expectation. Outfit: T-shirt and jeans, with a camera bag on her shoulder. Hair: long, neatly tied back. Posture: standing, holding a high-e

In [483]:
flat_clips = flatten_array(clips)
print_clip(flat_clips[9])

Scene number:  Scene 4
Clip number:  Clip 3
Time and background:  같은 장소, 사건 직후
Specific location description:  번화가 한복판, 사람들이 원을 그리며 상황을 지켜보는 모습
Characters:  주인공, 동기들, 구경꾼들
Detailed character description:  주인공: 바닥에 주저앉아 멍한 표정으로 일진의 뒷모습을 바라보는 모습, 눈물이 고임
동기들: 주인공을 걱정스럽게 바라보며 일으켜 세우려는 모습
구경꾼들: 동정 어린 시선으로 상황을 지켜보는 모습
Main action:  주인공의 내적 독백과 자아 성찰의 순간
Camera angle:  주인공의 얼굴을 클로즈업한 후 천천히 줌아웃하여 전체 상황을 와이드 샷으로 보여줌
Location atmosphere and emotion:  주인공의 수치심과 좌절감이 극에 달한 무거운 분위기
Event and key points:  주인공의 자아 성찰과 현실 직면
Narrations:  '엄청 쪽팔렸어. 그리고 난 아직 고등학생 이구나.. 라는 생각이 들었지.'
----------------------------------------------------------------------------------------------------


In [ ]:
def reformat_tts_script(dramatic_narration):
    return dramatic_narration.replace("\'", "\"")

print(reformat_tts_script(dramatic_narration))